In [ ]:
import glob
import os
from os import path

# Third-party
from astropy.constants import G
from astropy.io import fits, ascii
from astropy.stats import median_absolute_deviation
from astropy.table import Table, QTable, join
import astropy.units as u
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import numpy as np
%matplotlib inline
import tqdm
from scipy.stats import beta, binned_statistic
import h5py
import yaml

from thejoker import JokerParams, JokerSamples, RVData

from twoface.samples_analysis import MAP_sample
from twoface.plot import plot_two_panel, plot_phase_fold, plot_data_orbits, plot_phase_fold_residual
from twoface.mass import get_m2_min
from twoface.ext import FileLoader
from twoface.util import config_to_jokerparams

from apogeebh.plot import plot_diag_panel

In [ ]:
data_files = glob.glob('../data/candidates/*.ecsv')
cache_path = ('../data/candidates/cache/')

In [ ]:
with open('../config/bh.yml', 'r') as f:
    config = yaml.load(f.read())
joker_pars = config_to_jokerparams(config)

In [ ]:
sheet = FileLoader.GoogleSheets('1z0ukn8QWJL7dTrPdCFWlhQ6r-X-kwiA4z0YRqalZsY4', 1633658731)
candidates = sheet.load()

In [ ]:
def load_data_ecsv(fn):
    tbl = QTable.read(fn)
    data = RVData(t=tbl['time'],
                  rv=tbl['rv'],
                  stddev=tbl['rv_err'])
    return data, tbl['source']

In [ ]:
for data_fn in sorted(data_files):
    if '0618' not in data_fn: continue
    apogee_id = path.splitext(path.basename(data_fn))[0]
    samples_fn = path.join(cache_path, '{0}-joker.hdf5'.format(apogee_id))
    
    if apogee_id not in candidates['APOGEE_ID']:
        continue
        
    M1 = candidates[candidates['APOGEE_ID'] == apogee_id]['M1'] * u.Msun
    
    data, src = load_data_ecsv(data_fn)
    with h5py.File(samples_fn) as f:
        samples = JokerSamples.from_hdf5(f)
    
    fig = plot_diag_panel(data, samples, M1=M1, joker_params=joker_pars, data_src=src)
    fig.tight_layout()
    fig.savefig('../plots/{0}-joker.png'.format(apogee_id), dpi=250)
    plt.close(fig)